In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import pearsonr
import re

nhl_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

big4 = 'MLB'

In [2]:
sports = ['Metropolitan area', 'Population', 'NFL','MLB','NBA','NHL']
cities.columns = sports #rename the cities database columns
def cleaning(item):
    #print('working on function 1, string: {}'.format(item))
    s = re.sub(r'([a-z])([A-Z])', r'\1 \2', item) #separate two words by the capital letter e.g ColdCase Cold Case
    return s

def cleaning_2(item):
    #print('working on function 2, string: {}'.format(item))
    s = re.sub('\[.*\]', '', item) #remove the squared bracket
    #s = re.sub(r'([a-z])([A-Z])', r'\1 \2', item)
    return s

def cleaning_3(item): #clean the ends
    #print('working on function 3, string: {}'.format(item))
    item.strip()
    s = re.sub('\**\s\([0-9]*\)$', '', item)
    return s


def apply_cleaning(clen_fun, dataframe, columns): #applies all the cleaning functions on every col  
    for fun in clen_fun:
        for col in columns:
            dataframe[col]  = dataframe[col].astype('string').apply(fun)
    return dataframe

functions = [cleaning, cleaning_2, cleaning_3]

apply_cleaning(functions, cities, sports) # clean cities
cities.sort_values('Metropolitan area', axis = 0, inplace=True) #sort cities by their name
cities.reset_index(drop=True,inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,Atlanta,5789700,Falcons,Braves,Hawks,
1,Baltimore,2798886,Ravens,Orioles,,—
2,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
3,Buffalo,1132804,Bills,,,Sabres
4,Calgary,1392609,—,—,—,Flames


In [3]:
city_replace = {'Dallas–Fort Worth':'Dallas','Miami–Fort Lauderdale':'Miami','Minneapolis–Saint Paul':'Minneapolis','New York City':'New York','San Francisco Bay Area':'San Francisco','Tampa Bay Area':'Tampa Bay','Washington, D.C.':'Washington','':'None','—':'None','— ':'None'}
#row above does some name cleaning for the cities
cities.replace(to_replace=city_replace,value = None, inplace=True)
cities #this is the cleaned database for cities and their teams

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,Atlanta,5789700,Falcons,Braves,Hawks,None
1,Baltimore,2798886,Ravens,Orioles,None,None
2,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
3,Buffalo,1132804,Bills,None,None,Sabres
4,Calgary,1392609,None,None,None,Flames
5,Charlotte,2474314,Panthers,None,Hornets,None
6,Chicago,9512999,Bears,Cubs White Sox,Bulls,Blackhawks
7,Cincinnati,2165139,Bengals,Reds,None,None
8,Cleveland,2055612,Browns,Indians,Cavaliers,None
9,Columbus,2041520,None,None,None,Blue Jackets


In [4]:
nhl_df = nhl_df[nhl_df['year'] == 2018] #take only year 2018
#nhl_df.drop([0,9,18,26], axis = 0, inplace= True)
nhl_df = nhl_df[['team','W','L',]] #take only the important stuff
print(nhl_df.shape)
replace_nhl = {'Arizona':'Phoenix','Colorado':'Denver','Minnesota':'Minneapolis','Oakland':'San Francisco','Texas':'Dallas','Toronto Blue':'Toronto','Jays':'Blue Jays'}

(30, 3)


In [5]:
apply_cleaning(functions, nhl_df, list(nhl_df.columns)) #clean the database

,team,W,L
0,Boston Red Sox,108,54
1,New York Yankees,100,62
2,Tampa Bay Rays,90,72
3,Toronto Blue Jays,73,89
4,Baltimore Orioles,47,115
5,Cleveland Indians,91,71
6,Minnesota Twins,78,84
7,Detroit Tigers,64,98
8,Chicago White Sox,62,100
9,Kansas City Royals,58,104


In [6]:
def splitname(row):
    if 'Red Sox' in row['team']:
        row['Team'] = 'Red Sox'
        row['City'] = 'Boston'
        return row
    if 'White Sox' in row['team']:
        row['Team']  = 'White Sox'
        row['City'] = 'Chicago'
        return row
    else:
        row['Team']=row['team'].split(" ")[-1]
        s=row['team'].split(" ")[:-1]
        row['City']=' '.join(map(str, s)) #make it a single string instead of a list
        return row

nhl_df=nhl_df.apply(splitname, axis='columns')  #separe teams from cities
del(nhl_df['team'])
nhl_df

,W,L,Team,City
0,108,54,Red Sox,Boston
1,100,62,Yankees,New York
2,90,72,Rays,Tampa Bay
3,73,89,Jays,Toronto Blue
4,47,115,Orioles,Baltimore
5,91,71,Indians,Cleveland
6,78,84,Twins,Minnesota
7,64,98,Tigers,Detroit
8,62,100,White Sox,Chicago
9,58,104,Royals,Kansas City


In [7]:
nhl_df.replace(to_replace=replace_nhl,value=None,inplace=True) #clean "special" teams' names
nhl_df = nhl_df.set_index('City')
nhl_df.sort_values('City', axis=0, inplace=True)
nhl_df

,W,L,Team
City,,,
Atlanta,90,72,Braves
Baltimore,47,115,Orioles
Boston,108,54,Red Sox
Chicago,95,68,Cubs
Chicago,62,100,White Sox
Cincinnati,67,95,Reds
Cleveland,91,71,Indians
Dallas,67,95,Rangers
Denver,91,72,Rockies


In [8]:
cities_nhl = cities[cities[big4] != 'None'] #take all the cities that have a "NBA" team
cities_nhl = cities_nhl[['Metropolitan area','Population',big4]] # filter only interesting parameters
cities_nhl = cities_nhl.set_index('Metropolitan area') 
cities_nhl.index.rename('City', inplace=True) 
print('nr of cities: {}, nr of {} teams: {}'.format(cities_nhl.shape[0],big4,nhl_df.shape[0]))
cities_nhl.head()

nr of cities: 26, nr of MLB teams: 30


,Population,MLB
City,,
Atlanta,5789700,Braves
Baltimore,2798886,Orioles
Boston,4794447,Red Sox
Chicago,9512999,Cubs White Sox
Cincinnati,2165139,Reds


### This passage here is to check for manual cleaning

In [9]:
#cities_nhl[~cities_nhl.index.isin(nhl_df.index)] #show cities that are not in the NBA dataframe
#nhl_df[~nhl_df.index.isin(cities_nhl.index)] #Check what cities are not matching the names

In [10]:
cities_nhl = cities_nhl.merge(nhl_df, left_index=True, right_index=True)
cities_nhl.drop(labels=big4, axis = 1, inplace=True)
cities_nhl['W/L Ratio'] = cities_nhl['W'].astype('float64')/(cities_nhl['L'].astype('float64') + cities_nhl['W'].astype('float64'))
print(cities_nhl.shape)
cities_nhl


(30, 5)


,Population,W,L,Team,W/L Ratio
City,,,,,
Atlanta,5789700,90,72,Braves,0.555556
Baltimore,2798886,47,115,Orioles,0.290123
Boston,4794447,108,54,Red Sox,0.666667
Chicago,9512999,95,68,Cubs,0.582822
Chicago,9512999,62,100,White Sox,0.382716
Cincinnati,2165139,67,95,Reds,0.413580
Cleveland,2055612,91,71,Indians,0.561728
Dallas,7233323,67,95,Rangers,0.413580
Denver,2853077,91,72,Rockies,0.558282


In [11]:
cities_nhl['Avg. Ratio'] = cities_nhl.groupby(cities_nhl.index).agg(np.mean)
cities_nhl['Population'] = cities_nhl['Population'].astype('int64')
corr1 = cities_nhl['Population'].corr(cities_nhl['Avg. Ratio'],method='pearson')
print('answer to question 1: {}'.format(corr1))

answer to question 1: 0.19149307167656843


In [12]:
ans_df = cities_nhl[['Population','Avg. Ratio']]
ans_df.drop_duplicates(inplace=True)

In [13]:
population_by_region = list(ans_df['Population'])
win_loss_by_region = list(ans_df['Avg. Ratio'])

In [14]:
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.15027698302669307

In [15]:
population_by_region

[5789700,
 2798886,
 4794447,
 9512999,
 2165139,
 2055612,
 7233323,
 2853077,
 4297617,
 6772470,
 2104509,
 13310447,
 6066387,
 1572482,
 3551036,
 20153634,
 6070500,
 4661537,
 2342299,
 3317749,
 6657982,
 3798902,
 2807002,
 3032171,
 5928040,
 6131977]

In [16]:
len(win_loss_by_region)

26

In [17]:
win_loss_by_region

[0.5555555555555556,
 0.29012345679012347,
 0.6666666666666666,
 0.48276906763614325,
 0.41358024691358025,
 0.5617283950617284,
 0.41358024691358025,
 0.558282208588957,
 0.3950617283950617,
 0.6358024691358025,
 0.35802469135802467,
 0.5291221692039687,
 0.391304347826087,
 0.588957055214724,
 0.48148148148148145,
 0.5462962962962963,
 0.49382716049382713,
 0.5061728395061729,
 0.5093167701863354,
 0.4074074074074074,
 0.5246913580246914,
 0.5493827160493827,
 0.5432098765432098,
 0.5555555555555556,
 0.4506172839506173,
 0.5061728395061729]

In [18]:
cities_nhl.to_csv('assets/cleaned_'+big4+'.csv')